## EXPLORATORY DATA ANALYSIS

In [5]:
#Installing and importing the dependency
!pip install timm

In [6]:
!pip show timm

You may come across the question why i am installing timm in image classification problem i am going use efficient so i am installing that.

In [7]:
import timm

In [8]:
from fastai.vision.all import *

In [9]:
train_df = pd.read_csv("../input/paddy-disease-classification/train.csv")
train_df.sample(5)

In [10]:
train_df.variety.value_counts()

In [11]:
train_df["label"].value_counts()

In [12]:
train_df.info()

In [13]:
train_df.iloc[0]

In [14]:
train_images = get_image_files("../input/paddy-disease-classification/train_images")
train_images

In [15]:
train_images[0]

In [16]:
samp_sub = pd.read_csv("../input/paddy-disease-classification/sample_submission.csv")
samp_sub.sample()

In [17]:
data_dir = Path("../input/paddy-disease-classification")
train_dir = Path("../input/paddy-disease-classification/train_images")
test_dir = Path("../input/paddy-disease-classification/test_images")

In [18]:
data_dir.ls()

In [19]:
(train_dir/"tungro").ls()

In [20]:
Path.BASE_PATH = train_dir

In [21]:
train_dir.ls()

In [22]:
(train_dir/"hispa").ls()

In [23]:
test_dir.ls()

## DATA PREPARATION

I dont use the train_df for labels because paddy images are put in a folder with name, so i am going to use it.

In [24]:
paddy = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=RandomSplitter(seed=42),
                 get_y= parent_label,
                 item_tfms=Resize(460),
                 batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = paddy.dataloaders(train_dir)

Checking our dataloaders are build good aor bad by checking train and valid show batch.

In [25]:
dls.valid.show_batch(max_n=10, nrows=2)

In [26]:
dls.train.show_batch(max_n=10, nrows=2)

In [27]:
paddy.summary(train_dir)

I was using the model architecture like efficientnetB0, resnet34, resnet18, sometime using SGD or Adam, Everthing about trying let's try thats my mind says.

In [31]:
learn = vision_learner(dls, "efficientnet_b0", metrics=error_rate, opt_func=Adam)

In [32]:
lr_min,lr_steep = learn.lr_find(suggest_funcs=(minimum, steep))
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [33]:
learn.fine_tune??

In [35]:
learn = vision_learner(dls, "efficientnet_b0", metrics=error_rate, opt_func=Adam)
learn.fit_one_cycle(3, 9.12e-03)
learn.unfreeze()

In [36]:
lr_min,lr_steep = learn.lr_find(suggest_funcs=(minimum, steep))
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [37]:
learn.fit_one_cycle(12, lr_max=slice(1e-6,1e-4))

In [38]:
learn1 = vision_learner(dls, resnet34, metrics=error_rate)
learn1.fit_one_cycle(3, 3e-3)
learn1.unfreeze()
learn1.fit_one_cycle(12, lr_max=slice(1e-6,1e-4))

In [ ]:
from fastai.callback.fp16 import *
learn2 = vision_learner(dls, resnet18, metrics=error_rate).to_fp16()
learn2.fine_tune(6, freeze_epochs=3)

In [40]:
samp_sub.head()

In [41]:
test = test_dir.ls()
test


In [42]:
dl = learn.dls.test_dl(test)

In [43]:
dl

In [44]:
pred = learn.get_preds(dl=dl)

In [45]:
pred

In [46]:
preds = learn.get_preds(dl=dl)[0].argmax(1).numpy()
preds[:5]

In [47]:
train_df

In [48]:
samp_sub

In [49]:
preds

In [50]:
sub = pd.DataFrame({'image_id':samp_sub.image_id, 'label': preds})
sub.head()

In [51]:
labels = os.listdir('../input/paddy-disease-classification/train_images')
labels

In [53]:
ss = pd.read_csv('../input/paddy-disease-classification/sample_submission.csv')
ss['label'] = preds
ss['label'] = ss['label'].replace([0,1,2,3,4,5,6,7,8,9], labels)
ss.to_csv("EfficientB0.csv",index=False)
ss.head()

In [54]:
dl = learn1.dls.test_dl(test)

In [56]:
pred1 = learn1.get_preds(dl=dl)

In [57]:
pred1 = learn1.get_preds(dl=dl)[0].argmax(1).numpy()

In [58]:
ss1 = pd.read_csv('../input/paddy-disease-classification/sample_submission.csv')
ss1['label'] = pred1
ss1['label'] = ss['label'].replace([0,1,2,3,4,5,6,7,8,9], labels)
ss1.to_csv("resnet34.csv",index=False)
ss1.head()

In [59]:
dl = learn2.dls.test_dl(test)

In [60]:
pred2 = learn2.get_preds(dl=dl)

In [61]:
pred2 = learn2.get_preds(dl=dl)[0].argmax(1).numpy()

In [62]:
ss1 = pd.read_csv('../input/paddy-disease-classification/sample_submission.csv')
ss1['label'] = pred2
ss1['label'] = ss['label'].replace([0,1,2,3,4,5,6,7,8,9], labels)
ss1.to_csv("resnet18.csv",index=False)
ss1.head()